This is the code for random forest training by Raymond Xu.

 In this edition, the longitude and latitude of the original wildfire dataset is used as inputs.

 Here the temporal sequence is not considered in the data splitting.
 

In [2]:
''' This is Research Project titled ML Algorithms for Alberta Forest Occurence Prediction.
    This is the 8th Engineering Research Project, and is hereby delcared as
                            
                            Project Christopher
    
    Version 2.1 - Random Forest Classifier - All Parameters 
    Data Source: European Space Agency - ERA5
                 Government of Alberta - Historical Wildfire registry and Fire Weather Indices
                 Natural Resources Canada - Vegetation Classification of Canada
    
    AI Diederik - Hongik Ingan, For the Benefits of All Mankind
'''

import math
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.feature_selection import SelectKBest, f_classif,chi2
set_config(transform_output = "pandas")

Importing dada from Google Drive

In [3]:

# Load wildfire and non-wildfire datasets
wf_df = pd.read_csv("G:/Shared drives/MECE 788 - Forest Fire Prediction/04_Preprocessing/Cleanup_and_point_selection/wf_original lon and lat.csv",index_col=0)
main_df = pd.read_csv("G:/Shared drives/MECE 788 - Forest Fire Prediction/04_Preprocessing/Cleanup_and_point_selection/downsampled_df.csv",index_col=0)

# Remove the first unnamed column
wf_df=wf_df.iloc[: , 1:]

# Drop ERA5 lat and lon columns
wf_df=wf_df.drop(columns={'latitude_ERA5','longitude_ERA5'})
nwf_df=main_df[main_df['fire']==0].rename({
    'latitude_ERA5':'latitude',
    'longitude_ERA5':'longitude'
})


In [4]:
combined_data.keys()

NameError: name 'combined_data' is not defined

Set up the pipeline

In [29]:
# Manual separate numerical and categorical columns
pass_features = ['high_vegetation_cover',
       'low_vegetation_cover','type_of_high_vegetation', 'type_of_low_vegetation']
numeric_features = combined_data.drop(columns=pass_features).keys().drop(['fire'])
numeric_features = numeric_features.drop(['24hr_max_temperature'])
#numeric_features = numeric_features.insert(-1,'distance_to_road') Resevered for future development
selected_features ={'reserved'}#Columns to keep

# Define numeric and categorical transformer below
scale=ColumnTransformer([('scale_transformer',StandardScaler(),numeric_features)],verbose_feature_names_out=False).set_output(transform='pandas')

#cate=ColumnTransformer([('categorical_transformer',OneHotEncoder(sparse_output=False),categorical_features)],verbose_feature_names_out=False).set_output(transform='pandas')
#suspending onehote encoder
pss=ColumnTransformer([('Pass_transformer','passthrough',pass_features)],verbose_feature_names_out=False).set_output(transform='pandas')
Drop_transformer=ColumnTransformer([('Drop_transformer','passthrough',selected_features)],verbose_feature_names_out=False).set_output(transform='pandas')

Data_pipeline = Pipeline(steps=[
    ('Feature Union',FeatureUnion([('numeric', scale),('pass',pss)]))
    ]
    )
'''('Drop Columns',Drop_transformer)'''

"('Drop Columns',Drop_transformer)"

In [30]:
Data_pipeline

Pipeline(steps=[('Feature Union',
                 FeatureUnion(transformer_list=[('numeric',
                                                 ColumnTransformer(transformers=[('scale_transformer',
                                                                                  StandardScaler(),
                                                                                  Index(['latitude', 'longitude', 'leaf_area_index_high_vegetation',
       'leaf_area_index_low_vegetation', 'slope_of_sub_gridscale_orography',
       '24hr_accumulated_precipitation', 'global_noon_LST_2m_temperature',
       'global_...
       'global_noon_LST_2m_temperature_2dayLag', 'fire_1dayShift',
       'fire_2dayShift', 'fire_3dayShift'],
      dtype='object'))],
                                                                   verbose_feature_names_out=False)),
                                                ('pass',
                                                 ColumnTransformer(transformers=[('Pass_transformer',
                                                                                  'passthrough',
                                                                                  ['high_vegetation_cover',
                                                                                   'low_vegetation_cover',
                                                                                   'type_of_high_vegetation',
                                                                                   'type_of_low_vegetation'])],
                                                                   verbose_feature_names_out=False))]))])

Now divide the training and testing dataset:

In [31]:
# Divide the dataset into training, testing, and validation sets
train_ratio = 0.7
test_ratio = 0.30

train_size = int(train_ratio * len(combined_data))
test_size = int(test_ratio * len(combined_data))

train_data = combined_data[:train_size]
test_data = combined_data[train_size:train_size+test_size]

# Select equal numbers of wildfire and non-wildfire data points for the training set
num_wildfires = train_data['fire'].sum()
num_non_wildfires = train_size - num_wildfires

wildfire_train = train_data[train_data['fire'] == 1].sample(n=num_wildfires, replace=False)
non_wildfire_train = train_data[train_data['fire'] == 0].sample(n=num_non_wildfires, replace=False)
wildfire_test = test_data[test_data['fire'] == 1]
non_wildfire_test = test_data[test_data['fire'] == 0]
print("Training fire set size:", len(wildfire_train))
print("Training non-fire set size:", len(non_wildfire_train))
print("Training fire set size:", len(wildfire_test))
print("Training non-fire set size:", len(non_wildfire_test))

train_data = pd.concat([wildfire_train, non_wildfire_train])

# Output sizes of training, testing, and validation sets
print('-------------------------------------')
print("Training set size:", len(train_data))
print("Testing set size:", len(test_data))

X_train=train_data.drop(columns={'fire'})
X_test=test_data.drop(columns={'fire'})
y_train = train_data['fire']
y_test = test_data['fire']


Training fire set size: 38804
Training non-fire set size: 14530
Training fire set size: 11954
Training non-fire set size: 10903
-------------------------------------
Training set size: 53334
Testing set size: 22857


In [32]:
X_train_processed=Data_pipeline.fit_transform(X_train)
X_test_processed=Data_pipeline.transform(X_test)

In [33]:
# SelectKB

SelectKB_train = SelectKBest(f_classif,k=10)
SelectKB_train.fit(pd.DataFrame(X_train_processed,columns=X_train_processed.columns),y_train)
feature_names=SelectKB_train.get_feature_names_out()
print('Based on the Selected KBest Algorithm, the following 10 features should be selected:')
print(feature_names)

# PCA Analysis
from sklearn.decomposition import PCA
pca = PCA()
X_pca = pca.fit_transform(X_train_processed)
total_explained_variance = pca.explained_variance_ratio_.cumsum()
n_over_90 = len(total_explained_variance[total_explained_variance >= .90])
n_to_reach_90 = X_train_processed.shape[1] - n_over_90 + 1
print('-------------PCA Analysis----------------')
print("Number of features in the original dataset: {}".format(X_train_processed.shape[1]))
print("Number of features: {}\tTotal Variance Explained: {}".format(n_to_reach_90, total_explained_variance[n_to_reach_90-1]))


Based on the Selected KBest Algorithm, the following 10 features should be selected:
['numeric__24hr_accumulated_precipitation'
 'numeric__global_noon_LST_2m_temperature'
 'numeric__global_noon_LST_relative_humidity' 'numeric__BUI'
 'numeric__DMC' 'numeric__FFMC' 'numeric__FWI' 'numeric__fire_1dayShift'
 'numeric__fire_2dayShift' 'numeric__fire_3dayShift']
-------------PCA Analysis----------------
Number of features in the original dataset: 25
Number of features: 9	Total Variance Explained: 0.9142393864094365


In [ ]:
# Initialize and train the random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


param_grid = {
    'n_estimators': [30,800],  # Number of trees in the forest
    'max_depth': [30,8],
    'max_features': ['sqrt','log2'],
    'min_samples_leaf': [2,10],
    'min_samples_split': [5,7]
}

grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring=f1,n_jobs=4,verbose=1)
grid_search.fit(X_train_processed, y_train)

NameError: name 'X_train_processed' is not defined

In [ ]:
# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print(f"F1 Score: {100*grid_search.best_score_:.2f}%")

Best parameters: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 800}
F1 Score: 82.56%


In [ ]:
best_classifier = grid_search.best_estimator_

# Predict on the validation set
y_test_pred = best_classifier.predict(X_test_processed)

# Evaluate the accuracy of the model on the validation set
accuracy = accuracy_score(y_test, y_test_pred)
print("Validation set accuracy:", accuracy)
print('f1 score: ',f1_score(y_test, y_test_pred))

Validation set accuracy: 0.799886249289058
f1 score:  0.814397013471839


Saving the best model weight

In [13]:
import pickle

model_export = 'RF_all parameters.pkl'

with open(model_export,'wb') as file:
    pickle.dump(best_classifier,file)

NameError: name 'best_classifier' is not defined